In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [5]:
run_name <- '1_p_factor_v1'
phenotype <- 'HC382'
refit <- T
lambda_idx <- 55

data_d <- file.path(
    '/oak/stanford/groups/mrivas/projects/PRS/private_output/20200908_PRS_map_test', 
    run_name, phenotype, ifelse(refit, '2_refit', '1_fit_w_val')
)


In [6]:
# input and parameters

#data_d <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/20200908_PRS_map_test'
phe_f <- '/scratch/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.20200828.phe.zst'
covariates       <- c('age', 'sex', paste0('PC', 1:10))
refit_split_strs <- c('non_british_white', 'african', 's_asian', 'e_asian')
sscore_f             <- file.path(data_d, 'results', sprintf('snpnet.lambda%d.sscore.zst', lambda_idx))
snpnet_BETAs_f       <- file.path(data_d, 'results', sprintf('snpnet.lambda%d.tsv', lambda_idx))
snpnet_covar_BETAs_f <- file.path(data_d, 'results', sprintf('snpnet.lambda%d.covars.tsv', lambda_idx))
family <- ifelse((startsWith(phenotype, 'INI') | startsWith(phenotype, 'QT_FC')), 'gaussian', 'binomial')

# output
eval_f <- file.path(data_d, 'snpnet.AUCp.eval.tsv')
plot_f <- file.path(data_d, 'snpnet.AUCp.plot.pdf')
percentile_f <- file.path(data_d, 'snpnet.AUCp.percentile.tsv')


In [7]:
source('/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/snpnet/helpers/snpnet_misc.R')

# read the raw phenotype file
fread(
    cmd=paste(cat_or_zcat(phe_f), phe_f,  '|', 'sed -e "s/^#//g"'),
    select=c('FID', 'IID', 'split', covariates, phenotype),
    colClasses = c('FID'='character', 'IID'='character'),
    data.table=F
) -> phe_df

if(refit){
    phe_df %>%
    mutate(
        split = if_else(split %in% c('train', 'val'), 'train_val', split)
    ) -> phe_df
}

# read PRS and covariate-based score
phe_df %>%
compute_phe_score_df(
    phenotype,
    str_replace_all(sscore_f, '__PHENOTYPE__', phenotype),
    str_replace_all(snpnet_covar_BETAs_f, '__PHENOTYPE__', phenotype),
    covariates, family, refit_split_strs
) -> phe_score_df


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [8]:
# evaluate the predictive performance
phe_score_df  %>%
eval_performance(
    phenotype,
    str_replace_all(snpnet_BETAs_f, '__PHENOTYPE__', phenotype),
    family
) -> eval_df

eval_df %>%
rename('#phenotype_name' = 'phenotype_name') %>%
fwrite(str_replace_all(eval_f, '__PHENOTYPE__', phenotype), sep='\t', na = "NA", quote=F)


In [9]:
# prepare data frames for the plots
phe_score_df %>%
filter(split == 'test') %>% drop_na(geno_score, phe) %>%
mutate(
    geno_score_percentile = rank(-geno_score) / n()
) -> plot_df

summary_plot_df <- plot_df %>%
compute_summary_df('geno_score_percentile', 'phe', family=family)

summary_plot_df %>%
rename('#l_bin' = 'l_bin') %>%
fwrite(percentile_f, sep='\t', na = "NA", quote=F)


Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(percentile_col)` instead of `percentile_col` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(phe_col)` instead of `phe_col` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [10]:
if(family == 'gaussian'){
    p1 <- plot_df %>% plot_PRS_vs_phe() +
    theme(legend.position=c(.1, .8))+
    labs(title = phenotype, y = phenotype)

    p2 <- summary_plot_df %>%
    plot_PRS_bin_vs_phe(mean(plot_df$phe))+
    labs(title = phenotype, y = phenotype)
}else if(family == 'binomial'){
    p1 <- plot_df %>% plot_PRS_binomial() +
    labs(title = phenotype, x = phenotype)

    p2 <- summary_plot_df %>% plot_PRS_bin_vs_OR() +
    labs(title = phenotype)
}else{
    stop(sprintf('%s family is not supported!', family))
}


Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(geno_score_col)` instead of `geno_score_col` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [11]:
g <- gridExtra::arrangeGrob(p1, p2, ncol=2)
ggsave(plot_f, g, width=12, height=6)
ggsave(str_replace(plot_f, '.pdf$', '.png'), g, width=12, height=6)
